In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
import os
import fnmatch
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())

#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    8.540000e+02    8.540000e+02    8.540000e+02     8.540000e+02   
mean     1.896430e-11    9.393433e-13    2.781030e-14     2.132337e-13   
std      7.424042e-13    1.298968e-13    1.142721e-14     1.618587e-14   
min      1.747460e-11    5.272000e-13    8.900000e-15     3.330000e-14   
25%      1.845932e-11    9.475250e-13    2.090000e-14     2.040000e-13   
50%      1.880170e-11    9.696500e-13    2.680000e-14     2.128000e-13   
75%      1.972447e-11    1.006150e-12    3.230000e-14     2.230750e-13   
max      2.040510e-11    1.066300e-12    1.290000e-13     2.717000e-13   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     8.540000e+02     8.540000e+02     8.540000e+02     8.540000e+02   
mean      7.159988e-14     4.235774e-13     8.372730e-13     7.525101e-13   
std       4.781502e-14     2.373541e-14     4.438861e-14     5.709287e-14   
min       1.270000e-14   

,mass 2.0_time,mass 4.0_time,mass 5.5_time,mass 12.0_time,mass 15.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,...,"Elapsed time, mass 4","Elapsed time, mass 5.5","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2023/01/23 16:25:46.616,2023/01/23 16:25:46.866,2023/01/23 16:25:47.366,2023/01/23 16:25:47.616,2023/01/23 16:25:47.866,2023/01/23 16:25:48.117,2023/01/23 16:25:48.367,2023/01/23 16:25:48.617,2023/01/23 16:25:48.868,2023/01/23 16:25:49.619,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,2023/01/23 16:25:52.941,2023/01/23 16:25:53.191,2023/01/23 16:25:53.441,2023/01/23 16:25:53.691,2023/01/23 16:25:53.941,2023/01/23 16:25:54.192,2023/01/23 16:25:54.442,2023/01/23 16:25:54.692,2023/01/23 16:25:54.943,2023/01/23 16:25:55.193,...,6.325,6.075,6.075,6.075,6.075,6.075,6.075,6.075,5.574,NaN
4,2023/01/23 16:25:59.266,2023/01/23 16:25:59.516,2023/01/23 16:25:59.766,2023/01/23 16:26:00.016,2023/01/23 16:26:00.266,2023/01/23 16:26:00.517,2023/01/23 16:26:00.767,2023/01/23 16:26:01.017,2023/01/23 16:26:01.268,2023/01/23 16:26:01.518,...,12.650,12.400,12.400,12.400,12.400,12.400,12.400,12.400,11.899,NaN
5,2023/01/23 16:26:05.591,2023/01/23 16:26:05.841,2023/01/23 16:26:06.091,2023/01/23 16:26:06.341,2023/01/23 16:26:06.591,2023/01/23 16:26:06.842,2023/01/23 16:26:07.092,2023/01/23 16:26:07.342,2023/01/23 16:26:07.593,2023/01/23 16:26:07.843,...,18.975,18.725,18.725,18.725,18.725,18.725,18.725,18.725,18.224,NaN
6,2023/01/23 16:26:11.916,2023/01/23 16:26:12.166,2023/01/23 16:26:12.416,2023/01/23 16:26:12.666,2023/01/23 16:26:12.916,2023/01/23 16:26:13.167,2023/01/23 16:26:13.417,2023/01/23 16:26:13.667,2023/01/23 16:26:13.918,2023/01/23 16:26:14.168,...,25.300,25.050,25.050,25.050,25.050,25.050,25.050,25.050,24.549,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,2023/01/23 17:55:16.541,2023/01/23 17:55:16.791,2023/01/23 17:55:17.041,2023/01/23 17:55:17.291,2023/01/23 17:55:17.541,2023/01/23 17:55:17.792,2023/01/23 17:55:18.042,2023/01/23 17:55:18.292,2023/01/23 17:55:18.543,2023/01/23 17:55:18.793,...,5369.925,5369.675,5369.675,5369.675,5369.675,5369.675,5369.675,5369.675,5369.174,NaN
852,2023/01/23 17:55:22.866,2023/01/23 17:55:23.116,2023/01/23 17:55:23.366,2023/01/23 17:55:23.616,2023/01/23 17:55:23.866,2023/01/23 17:55:24.117,2023/01/23 17:55:24.367,2023/01/23 17:55:24.617,2023/01/23 17:55:24.868,2023/01/23 17:55:25.118,...,5376.250,5376.000,5376.000,5376.000,5376.000,5376.000,5376.000,5376.000,5375.499,NaN
853,2023/01/23 17:55:29.191,2023/01/23 17:55:29.441,2023/01/23 17:55:29.691,2023/01/23 17:55:29.941,2023/01/23 17:55:30.191,2023/01/23 17:55:30.442,2023/01/23 17:55:30.692,2023/01/23 17:55:30.942,2023/01/23 17:55:31.193,2023/01/23 17:55:31.443,...,5382.575,5382.325,5382.325,5382.325,5382.325,5382.325,5382.325,5382.325,5381.824,NaN
854,2023/01/23 17:55:35.516,2023/01/23 17:55:35.766,2023/01/23 17:55:36.016,2023/01/23 17:55:36.266,2023/01/23 17:55:36.516,2023/01/23 17:55:36.767,2023/01/23 17:55:37.017,2023/01/23 17:55:37.268,2023/01/23 17:55:37.518,2023/01/23 17:55:37.768,...,5388.900,5388.650,5388.650,5388.650,5388.650,5388.650,5388.651,5388.650,5388.149,NaN


In [3]:
#read in the baseline MID

basel = pd.read_csv("baseline MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
basel.drop([0,1], inplace=True)
print(basel.describe())
#bring the time data into a new df
time1 = pd.DataFrame(basel.iloc[:,0::2]) #- get every second column
print(time.describe())

#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time1)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    4.930000e+02    4.930000e+02    4.930000e+02     4.930000e+02   
mean     2.241972e-11    1.068944e-12    3.190791e-14     9.334178e-14   
std      6.636432e-12    4.121569e-13    1.128018e-14     6.767923e-14   
min      1.291550e-11    3.786000e-13    1.000000e-16     4.800000e-15   
25%      1.344180e-11    5.117000e-13    2.610000e-14     5.710000e-14   
50%      2.682440e-11    1.351100e-12    3.120000e-14     6.530000e-14   
75%      2.773280e-11    1.387900e-12    3.560000e-14     7.510000e-14   
max      2.831060e-11    1.429800e-12    1.177000e-13     2.670000e-13   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     4.930000e+02     4.930000e+02     4.930000e+02     4.930000e+02   
mean      5.676978e-14     3.824718e-13     7.099882e-13     7.647097e-13   
std       3.371720e-14     5.538724e-14     4.742495e-14     1.009312e-13   
min       2.600000e-15   

,mass 2.0_time,mass 4.0_time,mass 5.5_time,mass 12.0_time,mass 15.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,...,"Elapsed time, mass 4","Elapsed time, mass 5.5","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2023/01/23 14:56:43.412,2023/01/23 14:56:43.662,2023/01/23 14:56:43.912,2023/01/23 14:56:44.162,2023/01/23 14:56:44.412,2023/01/23 14:56:44.662,2023/01/23 14:56:44.913,2023/01/23 14:56:45.163,2023/01/23 14:56:45.413,2023/01/23 14:56:45.664,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2023/01/23 14:56:49.737,2023/01/23 14:56:49.987,2023/01/23 14:56:50.237,2023/01/23 14:56:50.487,2023/01/23 14:56:50.737,2023/01/23 14:56:50.987,2023/01/23 14:56:51.238,2023/01/23 14:56:51.488,2023/01/23 14:56:51.738,2023/01/23 14:56:51.989,...,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.325
4,2023/01/23 14:56:56.062,2023/01/23 14:56:56.312,2023/01/23 14:56:56.562,2023/01/23 14:56:56.812,2023/01/23 14:56:57.062,2023/01/23 14:56:57.312,2023/01/23 14:56:57.563,2023/01/23 14:56:57.813,2023/01/23 14:56:58.064,2023/01/23 14:56:58.314,...,12.650,12.650,12.650,12.650,12.650,12.650,12.650,12.651,12.650,12.650
5,2023/01/23 14:57:02.387,2023/01/23 14:57:02.637,2023/01/23 14:57:02.887,2023/01/23 14:57:03.137,2023/01/23 14:57:03.387,2023/01/23 14:57:03.638,2023/01/23 14:57:03.888,2023/01/23 14:57:04.138,2023/01/23 14:57:04.389,2023/01/23 14:57:04.639,...,18.975,18.975,18.975,18.975,18.976,18.975,18.975,18.976,18.975,18.975
6,2023/01/23 14:57:08.712,2023/01/23 14:57:08.962,2023/01/23 14:57:09.212,2023/01/23 14:57:09.462,2023/01/23 14:57:09.712,2023/01/23 14:57:09.962,2023/01/23 14:57:10.213,2023/01/23 14:57:10.463,2023/01/23 14:57:10.714,2023/01/23 14:57:10.964,...,25.300,25.300,25.300,25.300,25.300,25.300,25.300,25.301,25.300,25.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2023/01/23 15:48:10.012,2023/01/23 15:48:10.262,2023/01/23 15:48:10.512,2023/01/23 15:48:10.762,2023/01/23 15:48:11.012,2023/01/23 15:48:11.263,2023/01/23 15:48:11.513,2023/01/23 15:48:11.763,2023/01/23 15:48:12.014,2023/01/23 15:48:12.264,...,3086.600,3086.600,3086.600,3086.600,3086.601,3086.600,3086.600,3086.601,3086.600,3086.600
491,2023/01/23 15:48:16.337,2023/01/23 15:48:16.587,2023/01/23 15:48:16.837,2023/01/23 15:48:17.087,2023/01/23 15:48:17.337,2023/01/23 15:48:17.588,2023/01/23 15:48:17.838,2023/01/23 15:48:18.088,2023/01/23 15:48:18.339,2023/01/23 15:48:18.589,...,3092.925,3092.925,3092.925,3092.925,3092.926,3092.925,3092.925,3092.926,3092.925,3092.925
492,2023/01/23 15:48:22.662,2023/01/23 15:48:22.912,2023/01/23 15:48:23.162,2023/01/23 15:48:23.412,2023/01/23 15:48:23.662,2023/01/23 15:48:23.913,2023/01/23 15:48:24.163,2023/01/23 15:48:24.413,2023/01/23 15:48:24.664,2023/01/23 15:48:24.914,...,3099.250,3099.250,3099.250,3099.250,3099.251,3099.250,3099.250,3099.251,3099.250,3099.250
493,2023/01/23 15:48:28.987,2023/01/23 15:48:29.237,2023/01/23 15:48:29.487,2023/01/23 15:48:29.737,2023/01/23 15:48:29.987,2023/01/23 15:48:30.238,2023/01/23 15:48:30.488,2023/01/23 15:48:30.738,2023/01/23 15:48:30.989,2023/01/23 15:48:31.239,...,3105.575,3105.575,3105.575,3105.575,3105.576,3105.575,3105.575,3105.576,3105.575,3105.575


In [4]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,23/01/2023 15:16:40,7.0,-0.062320,5.030000e-10,1.150000e-09,4.996000e-09,3.718000e-07,0.003782,0.0,-57.014,95.061,NaN,NaN
2,LHe transfer + inj line rinsing,23/01/2023 15:16:48,15.0,-0.062269,5.010000e-10,1.140000e-09,4.996000e-09,3.714000e-07,0.003772,0.0,-56.453,95.735,NaN,NaN
3,NaN,23/01/2023 15:16:55,22.0,-0.062737,4.970000e-10,1.140000e-09,4.996000e-09,3.718000e-07,0.003782,0.0,-56.004,96.184,NaN,NaN
4,NaN,23/01/2023 15:17:02,29.0,-0.062048,4.990000e-10,1.150000e-09,4.996000e-09,3.714000e-07,0.003744,0.0,-55.555,96.857,NaN,NaN
5,NaN,23/01/2023 15:17:10,37.0,-0.061381,4.990000e-10,1.140000e-09,4.996000e-09,3.714000e-07,0.003747,0.0,-54.994,97.418,NaN,NaN


In [5]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
#hv["I_em"]=abs(hv["I_em"])
#hv["hv_grid"]=abs(hv["hv_grid"])
hv["datetime"] = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
hv['datetime'] = hv['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
#hv.insert(0,"datetime",hv["Date"]+" "+hv["Time"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)


hv



,Date,Time,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,I_em,datetime
0,23/01/2023,16:25:25.943,0.000,0.02,0.08,0.06,0.10,0.00,0.00,0.0,...,-0.0011,0.0072,0.0061,0.0110,-0.0318,-0.0075,NaN,1.400000e-09,5.700000e-09,2023-01-23 16:25:25.943000
1,23/01/2023,16:25:27.130,1.187,0.02,0.08,0.06,0.10,0.00,0.00,0.0,...,-0.0011,0.0072,0.0060,0.0110,-0.0318,-0.0075,NaN,1.400000e-09,5.700000e-09,2023-01-23 16:25:27.130000
2,23/01/2023,16:25:28.295,2.352,0.02,0.08,0.06,0.10,0.00,0.00,0.0,...,-0.0011,0.0072,0.0060,0.0110,-0.0318,-0.0075,NaN,1.400000e-09,5.700000e-09,2023-01-23 16:25:28.295000
3,23/01/2023,16:25:29.463,3.520,0.02,0.08,0.06,0.10,0.00,0.00,0.0,...,-0.0011,0.0072,0.0060,0.0110,-0.0318,-0.0075,NaN,1.400000e-09,5.700000e-09,2023-01-23 16:25:29.463000
4,23/01/2023,16:25:30.628,4.685,0.02,0.08,0.06,0.10,0.00,0.00,0.0,...,-0.0011,0.0072,0.0060,0.0110,-0.0318,-0.0075,NaN,1.400000e-09,5.700000e-09,2023-01-23 16:25:30.628000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,23/01/2023,17:54:15.011,5329.068,499.98,100.10,1000.10,1000.26,1000.18,499.98,0.0,...,25.1156,25.1556,25.1208,25.1268,-0.0303,-0.0056,NaN,2.110000e-08,2.110000e-08,2023-01-23 17:54:15.011000
4433,23/01/2023,17:54:16.214,5330.271,499.98,100.10,1000.10,1000.26,1000.18,499.98,0.0,...,25.1206,25.1602,25.1245,25.1307,-0.0303,-0.0056,NaN,2.110000e-08,2.110000e-08,2023-01-23 17:54:16.214000
4434,23/01/2023,17:54:17.417,5331.474,499.98,100.10,1000.10,1000.26,1000.18,499.98,0.0,...,25.1214,25.1612,25.1258,25.1309,-0.0303,-0.0056,NaN,2.110000e-08,2.110000e-08,2023-01-23 17:54:17.417000
4435,23/01/2023,17:54:18.616,5332.673,499.98,100.10,1000.10,1000.26,1000.18,499.98,0.0,...,25.1197,25.1589,25.1237,25.1282,-0.0303,-0.0056,NaN,2.110000e-08,2.110000e-08,2023-01-23 17:54:18.616000


### Data processing - single point desorption

In [6]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [7]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)

#then find the time value closest to it in the MID scan file 
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][91:136]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][56:91]/h2_sens
base_h2.describe()

Result 1:             Date          Time  Elapsed time   VMon1   VMon2    VMon3  \
2751  23/01/2023  17:20:35.866      3309.923  439.56  100.10  1000.08   
2752  23/01/2023  17:20:37.062      3311.119  309.44  100.10  1000.08   
2753  23/01/2023  17:20:38.264      3312.321  178.90  100.08  1000.08   
2754  23/01/2023  17:20:39.466      3313.523   89.12   99.96  1000.12   

        VMon4    VMon5   VMon6  VMon7  ...    IMon3    IMon4    IMon5  \
2751  1000.24  1000.20  499.98    0.0  ...  26.9970  27.0368  27.0057   
2752  1000.24  1000.20  499.98    0.0  ...  26.9945  27.0345  27.0033   
2753  1000.24  1000.20  499.98    0.0  ...  26.9207  27.0224  26.9937   
2754  1000.26  1000.22  500.00    0.0  ...  26.5100  26.6910  26.7877   

        IMon6   IMon7   IMon8           Comment       hv_grid          I_em  \
2751  27.0030 -0.0322 -0.0071  grid transparent -4.339000e-07  2.080000e-08   
2752  27.0005 -0.0322 -0.0071  grid transparent -4.438000e-07  2.710000e-08   
2753  26.9923 -0.0322

count    3.500000e+01
mean     9.356817e-10
std      4.720008e-12
min      9.271194e-10
25%      9.322509e-10
50%      9.363541e-10
75%      9.390537e-10
max      9.473307e-10
Name: mass 2, corr, dtype: float64

## Point-by-point desorption

In [8]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]

#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                    datetime   Ion current
0    2023/01/23 16:25:46.616  1.923060e-11
1    2023/01/23 16:25:52.941  1.954450e-11
2    2023/01/23 16:25:59.266  1.957250e-11
3    2023/01/23 16:26:05.591  1.925910e-11
4    2023/01/23 16:26:11.916  1.951390e-11
..                       ...           ...
849  2023/01/23 17:55:16.541  1.749560e-11
850  2023/01/23 17:55:22.866  1.759290e-11
851  2023/01/23 17:55:29.191  1.764230e-11
852  2023/01/23 17:55:35.516  1.759660e-11
853  2023/01/23 17:55:41.841  1.753480e-11

[854 rows x 2 columns]


In [9]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)

#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2,left_index=True, right_index=True, how="inner")

print("aligned",hv_aligned)
print(hv_aligned.describe())

                           Date          Time  Elapsed time   VMon1   VMon2  \
datetime                                                                      
2023-01-23 16:25:26  23/01/2023  16:25:25.943         0.000    0.02    0.08   
2023-01-23 16:25:27  23/01/2023  16:25:27.130         1.187    0.02    0.08   
2023-01-23 16:25:28  23/01/2023  16:25:28.295         2.352    0.02    0.08   
2023-01-23 16:25:29  23/01/2023  16:25:29.463         3.520    0.02    0.08   
2023-01-23 16:25:31  23/01/2023  16:25:30.628         4.685    0.02    0.08   
...                         ...           ...           ...     ...     ...   
2023-01-23 17:54:15  23/01/2023  17:54:15.011      5329.068  499.98  100.10   
2023-01-23 17:54:16  23/01/2023  17:54:16.214      5330.271  499.98  100.10   
2023-01-23 17:54:17  23/01/2023  17:54:17.417      5331.474  499.98  100.10   
2023-01-23 17:54:19  23/01/2023  17:54:18.616      5332.673  499.98  100.10   
2023-01-23 17:54:20  23/01/2023  17:54:19.819      5

### From H2 ion current to partial pressure:

In [10]:
#Filtering background signal for resampled data

#take a mass with a signal known to be in the background as a definition of electronic noise
#Ignored because has negligeable impact (~1%)
#elec_bck = np.mean(mid["mass 5.5_value"])

# take the Ion current of H2 channel before emission start as a reference to base pressure:
#get the rows where elapsed time is <600s
pre_EM = hv_aligned.loc[hv_aligned["Elapsed time"] <= 600]
dyn_bck = np.mean(pre_EM["Ion current"])
print(dyn_bck)

#apply background corrections
hv_aligned["mass 2 corr"] = hv_aligned["Ion current"] - dyn_bck #- elec_bck 
#remove values below 0:
hv_aligned["mass 2 corr"].clip(0, inplace=True)

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #[A/mbar]
p_h2_ip = hv_aligned["mass 2 corr"]/(h2_sens*0.46) #H2 equivalent pressures

#account for the thermal transpiration factor (electrons thermalising to RT)
T2 = 4.2
T1 = 296.5
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(22,"hydrogen pp",p_h2_ip)
print(hv_aligned["hydrogen pp"].describe())

1.9883297826086954e-11
count    8.540000e+02
mean     2.548910e-13
std      7.797544e-13
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      6.338416e-12
Name: hydrogen pp, dtype: float64


In [11]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].describe()

count    8.540000e+02
mean     2.548910e-13
std      7.797544e-13
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      6.338416e-12
Name: hydrogen pp, dtype: float64

### Cut away the warmup part from datasets


In [12]:
#cut the hv_aligned dataframe

cut_hv = hv_aligned.loc[hv_aligned["Elapsed time"] <= 5000]
#cut the mid and time dataframes

cut_time = time.loc[time['Elapsed time, mass 2'] <= 5000]
#print(cut_time)

#slicing mid to match time ["Elapsed time, mass 2']:
# Slice x using the index of y
cut_mid = mid.loc[cut_time.index, :]

#remove the spike in emission current between elapsed time[1335:1367]
spike = cut_hv[cut_hv["Elapsed time"].between(1335,1367)]
print(cut_hv)
#find indexes that correspond to these values
#cut_hv.drop(cut_hv.loc[spike.index],inplace=True, axis=0)
cut_hv = cut_hv.loc[(cut_hv.index < '2023-1-16 15:51:53') | (cut_hv.index > '2023-01-16 15:52:24')]
print(cut_hv)


                           Date          Time  Elapsed time  VMon1  VMon2  \
datetime                                                                    
2023-01-23 16:25:47  23/01/2023  16:25:47.097        21.154   0.02   0.08   
2023-01-23 16:25:53  23/01/2023  16:25:53.000        27.057   0.02   0.08   
2023-01-23 16:25:59  23/01/2023  16:25:58.848        32.905   0.02   0.08   
2023-01-23 16:26:06  23/01/2023  16:26:05.899        39.956   0.02   0.08   
2023-01-23 16:26:12  23/01/2023  16:26:11.813        45.870   0.02   0.08   
...                         ...           ...           ...    ...    ...   
2023-01-23 17:55:17           0             0         0.000   0.00   0.00   
2023-01-23 17:55:23           0             0         0.000   0.00   0.00   
2023-01-23 17:55:29           0             0         0.000   0.00   0.00   
2023-01-23 17:55:36           0             0         0.000   0.00   0.00   
2023-01-23 17:55:42           0             0         0.000   0.00   0.00   

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [13]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 295.5
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000
#Emission current
#averaging
I_em = np.nanmean(hv["I_em"].nlargest(30))
print(I_em)



-2.92067963335587e-10
6.0685e-06


In [14]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: -3.762598873597166e-05 



## Desorption yield point-by-point

$$
\eta=\frac{C_{H2}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$


In [15]:
#partial pressure delta
#Wrong methodology below. Need to subtract p_duringEM form p_base, noEM
#p_delta = hv_aligned["hydrogen pp"].diff(periods=1).fillna(hv_aligned["hydrogen pp"])*100 #conversion to Pa
#p_delta = p_delta.clip(lower=0)

#Correct methodology
#for the base pressure use the first 600s from cut_hv["Ion current"]
no_EM = cut_hv.loc[cut_hv["Elapsed time"] <= 600]
print(no_EM.describe())
mean_pbase = np.mean(no_EM["hydrogen pp"])

print(mean_pbase)

EM = cut_hv.loc[cut_hv["Elapsed time"] >= 600]
p_delta = (EM["hydrogen pp"] - mean_pbase)*100
p_delta.clip(0, inplace=True)
print(p_delta)

       Elapsed time       VMon1       VMon2        VMon3        VMon4  \
count    104.000000  104.000000  104.000000   104.000000   104.000000   
mean     272.991048  278.816154   54.894615   531.837500   516.524038   
std      187.114514  249.492757   50.033908   499.264833   499.370330   
min        0.000000    0.000000    0.000000     0.000000     0.000000   
25%      108.104250    0.020000    0.080000     0.080000     0.100000   
50%      270.653500  499.900000  100.100000   999.730000   999.690000   
75%      433.842250  499.980000  100.100000  1000.000000  1000.060000   
max      595.654000  500.000000  100.160000  1000.060000  1000.140000   

            VMon5       VMon6  VMon7  VMon8       IMon1  ...       IMon4  \
count   104.00000  104.000000  104.0  104.0  104.000000  ...  104.000000   
mean    502.85750  244.307115    0.0    0.0    0.013610  ...   13.295836   
std     500.33856  250.401964    0.0    0.0    0.018208  ...   12.822786   
min       0.00000    0.000000    0.0  

In [16]:
#constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
I_n = EM["I_em"].replace(0, np.nan)
I_n.where(EM["I_em"] >= 9e-7, np.nan, inplace=True) #--- this will cause misalignment if NaNs are thrown away
I_n.dropna(inplace=True)
#pd.df.where(cond= "Where cond is True, keep the original value. Otherwise, replace with NaN")
print(I_n.describe())

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
#ignore first row to match "i" dataset
eta = eta.loc[eta.index[1]:]

print(spike)
#eta = eta
print("desorption yields point-by-point:","\n", eta)

count    3.870000e+02
mean     5.625795e-06
std      2.894855e-07
min      5.083500e-06
25%      5.413850e-06
50%      5.587400e-06
75%      5.927800e-06
max      6.192600e-06
Name: I_em, dtype: float64
                           Date          Time  Elapsed time  VMon1   VMon2  \
datetime                                                                     
2023-01-23 16:47:42  23/01/2023  16:47:42.218      1336.275   0.18  100.10   
2023-01-23 16:47:49  23/01/2023  16:47:49.508      1343.565   0.18  100.10   
2023-01-23 16:47:55  23/01/2023  16:47:55.543      1349.600   0.20  100.10   
2023-01-23 16:48:01  23/01/2023  16:48:01.543      1355.600   0.20  100.08   
2023-01-23 16:48:08  23/01/2023  16:48:07.584      1361.641   0.20  100.08   

                       VMon3   VMon4   VMon5   VMon6  VMon7  ...    IMon5  \
datetime                                                     ...            
2023-01-23 16:47:42  1000.06  1000.2  1000.2  500.02    0.0  ...  28.5249   
2023-01-23 16:47:49

In [17]:
#slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :]
hv_downsampled.describe()

,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,...,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,I_em,hydrogen pp,Ion current,mass 2 corr
count,386.000000,386.000000,386.000000,386.000000,386.000000,386.000000,386.000000,386.0,386.0,386.000000,...,386.000000,386.000000,386.000000,386.000000,386.000000,3.860000e+02,3.860000e+02,3.860000e+02,3.860000e+02,3.860000e+02
mean,2894.273461,0.156943,100.093109,1000.087461,1000.223834,1000.196425,499.987927,0.0,0.0,-0.830516,...,27.244305,27.212753,27.210832,-0.030935,-0.006853,-8.305158e-07,5.624531e-06,2.485384e-13,1.905816e-11,2.046061e-14
std,1304.617798,0.024925,0.011404,0.016757,0.026459,0.012676,0.011638,0.0,0.0,0.056942,...,1.132713,1.134976,1.133793,0.001199,0.001414,5.694187e-08,2.887912e-07,8.332430e-13,6.468177e-13,6.859568e-14
min,943.917000,0.120000,99.980000,1000.060000,1000.160000,1000.160000,499.960000,0.0,0.0,-0.946300,...,25.690000,25.656100,25.654600,-0.032900,-0.010100,-9.463000e-07,5.083500e-06,0.000000e+00,1.808310e-11,0.000000e+00
25%,1552.999500,0.140000,100.080000,1000.080000,1000.200000,1000.180000,499.980000,0.0,0.0,-0.888575,...,26.157800,26.123625,26.126625,-0.032200,-0.007700,-8.885750e-07,5.413575e-06,0.000000e+00,1.846577e-11,0.000000e+00
50%,3464.771000,0.160000,100.100000,1000.080000,1000.220000,1000.200000,499.980000,0.0,0.0,-0.821200,...,26.820550,26.787850,26.784050,-0.030900,-0.006400,-8.212000e-07,5.587200e-06,0.000000e+00,1.875840e-11,0.000000e+00
75%,4073.995000,0.180000,100.100000,1000.100000,1000.240000,1000.200000,500.000000,0.0,0.0,-0.783250,...,28.516375,28.485650,28.484075,-0.030100,-0.005700,-7.832500e-07,5.926500e-06,0.000000e+00,1.968995e-11,0.000000e+00
max,4682.585000,0.200000,100.120000,1000.120000,1000.260000,1000.220000,500.020000,0.0,0.0,-0.733100,...,28.784200,28.752000,28.750800,-0.028500,-0.005000,-7.331000e-07,6.192600e-06,4.920842e-12,2.028840e-11,4.051022e-13


## Electron dose


Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [18]:
#Integrating the emission current

def integrand(t, i):
    return i*t

i = I_n.dropna().reset_index() 

#for testing:
#t2_2 = (hv_aligned["Elapsed time"].diff(periods=1).fillna(0))

#adding an elapsed time column
timestamp3=[]
for j in hv["Time"]:
    timestamp3.append(j)
    
#making the column into a datetime object
timestamps3=pd.to_datetime(i["datetime"],format="%H:%M:%S")
runtime3=(timestamps3-timestamps3[0]).dt.total_seconds()    
i.insert(2,"Elapsed time",runtime3)

#define the n, n-1 difference in a variable "t2"
t2 = (i["Elapsed time"].diff(periods=1).fillna(0))

#replace all values from t2 that are further away than 50% of the median (.. > 9) with the median value
t2.where(t2 <= 12,6,inplace=True)
print(t2)

#integration
ar = []
for x in t2:
    itr = 0
    I = quad(integrand, 0, x, args=(i["I_em"][itr]))
    ar.append(I[0])
    itr += 1
    #if previous value was not NaN, replace the value with the median of elapsed time
    #if 
    

0      0.0
1      6.0
2      6.0
3      7.0
4      6.0
      ... 
382    6.0
383    6.0
384    6.0
385    7.0
386    6.0
Name: Elapsed time, Length: 387, dtype: float64


In [19]:
#calculate the cumulative charge
Q_cumul = np.cumsum(ar)
Q_sum = sum(ar)
print("Total charge emitted from EGA during ~5000 seconds:", f"{Q_sum:.4E}", "C")

#cumulative nr of electrons emitted
N_e = Q_sum/1.6e-19
Ne_cumul = Q_cumul/1.6e-19
print("Total number of electron emitted during ~5000 seconds:", f"{N_e:.4E}", "e")
      
#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9
tot_dose = (N_e/276)*0.9
cumul_dose = (Ne_cumul/276)*0.9
print("Total electron dose on the sample ~5000 seconds:", f"{tot_dose:.4E}", "e-/cm^2")
      
#add these to the "i" dataframe:
i.insert(3,"Cumulative charge", Q_cumul)
i.insert(4,"Cumulative n. of electrons emitted from EGA", Ne_cumul)
i.insert(5,"Cumulative electron dose on the HL sample", cumul_dose)
i.set_index("datetime", drop=True, inplace=True)

#Ignore the first value, also, match the shape of eta
i = i.loc[i.index[1]:]
print(i)


Total charge emitted from EGA during ~5000 seconds: 4.7443E-02 C
Total number of electron emitted during ~5000 seconds: 2.9652E+17 e
Total electron dose on the sample ~5000 seconds: 9.6691E+14 e-/cm^2
                         I_em  Elapsed time  Cumulative charge  \
datetime                                                         
2023-01-23 16:41:10  0.000006           6.0           0.000110   
2023-01-23 16:41:16  0.000006          12.0           0.000220   
2023-01-23 16:41:23  0.000006          19.0           0.000370   
2023-01-23 16:41:29  0.000006          25.0           0.000480   
2023-01-23 16:41:35  0.000006          31.0           0.000590   
...                       ...           ...                ...   
2023-01-23 17:43:03  0.000005        3719.0           0.046963   
2023-01-23 17:43:09  0.000005        3725.0           0.047073   
2023-01-23 17:43:15  0.000005        3731.0           0.047183   
2023-01-23 17:43:22  0.000005        3738.0           0.047333   
2023-01

# Plotting

In [20]:
#plot the aligned dataframe results

plt.figure(figsize=(12,6))
plt.plot(cut_hv["Elapsed time"],cut_hv["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(cut_hv["Elapsed time"],cut_hv["I_em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 2e18 M pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [21]:
#plot the h2 mid 
#seems to be too little rows?

plt.figure(figsize=(12,6))
plt.plot(cut_hv["Elapsed time"],cut_hv["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Ion current evolution of H2, 2e18 M pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('linear')


In [22]:
#plot the aligned dataframe results
fig, ax = plt.subplots()
ax.plot(cut_hv["Elapsed time"],cut_hv["I_em"],linestyle="-",marker=".",c="r", markersize=4,label='I_EM')
ax.set_xlim(left=140)

ax.set_xlabel('Elapsed time (s)')
ax.set_ylabel('Current (A)')
ax.set_yscale('log')
ax.legend(loc="upper right")
ax.axes.grid(visible=None,axis="y")
ax2 = ax.twinx()
ax2.plot(cut_hv["Elapsed time"],cut_hv["hydrogen pp"],linestyle="-",marker=".", c="b",markersize=4,label='H2 partial pressure')
ax2.set_ylabel('Pressure (mbar)')
ax2.set_yscale('linear')
ax2.legend(loc="upper left")

plt.title("HiLumi sample no signal @4K no pre-coverage")

Text(0.5, 1.0, 'HiLumi sample no signal @4K no pre-coverage')

In [23]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(cut_time['Elapsed time, mass 2'],cut_mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(cut_time['Elapsed time, mass 4'],cut_mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(cut_time['Elapsed time, mass 12'],cut_mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(cut_time['Elapsed time, mass 16'],cut_mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(cut_time['Elapsed time, mass 18'],cut_mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(cut_time['Elapsed time, mass 28'],cut_mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(cut_time['Elapsed time, mass 32'],cut_mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(cut_time['Elapsed time, mass 40'],cut_mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(cut_time["Elapsed time, mass 44"],cut_mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan 2e18 M pre-injection, EGA fully in, dynamic vacuum")

plt.yscale('log')

In [24]:
#Plotting the baseline mid
plt.figure(figsize=(14,8))
plt.plot(time1['Elapsed time, mass 2'],basel["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time1['Elapsed time, mass 4'],basel["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time1['Elapsed time, mass 12'],basel["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time1['Elapsed time, mass 16'],basel["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time1['Elapsed time, mass 18'],basel["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time1['Elapsed time, mass 28'],basel["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time1['Elapsed time, mass 32'],basel["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time1['Elapsed time, mass 40'],basel["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time1["Elapsed time, mass 44"],basel["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID baseline scan 2e18 M pre-injection, EGA fully in, dynamic vacuum")

plt.yscale('log')

In [25]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 2e18 M pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [26]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution 2e18 M pre-injection, EGA fully in, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [27]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution 2e18 M pre-injection, EGA fully in, dynamic vacuum")
plt.yscale('linear')

In [28]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el) 2e18 M pre-injection')
plt.legend()
plt.title("ESD vs time, p_n - p_n-1")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [29]:
#plotting ESD vs electron dose
plt.figure(figsize=(10,6))
plt.plot(i["Cumulative electron dose on the HL sample"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Electron dose (e/cm^2)')
plt.ylabel('ESD yield (M/el)')
plt.legend()
plt.title("Electron stimulated desorption yield vs electron dose")
ax = plt.gca()
ax.set_yscale('linear')
ax.grid(which="major",axis="both")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()